# ノック41

ノック41 - 50では、決定木を用いて退会会員の予測をしていく。

In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# データの読み込み

customer = pd.read_csv("customer_join.csv")
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [3]:
uselog_months = pd.read_csv("use_log_months.csv")
uselog_months.head()

,年月,customer_id,count
0,201804,AS002855,4
1,201804,AS009013,2
2,201804,AS009373,3
3,201804,AS015315,6
4,201804,AS015739,7


今回は1カ月前のデータから当月の退会予測を導き出す。  

In [4]:
year_months = list(uselog_months["年月"].unique())
year_months

[201804,
 201805,
 201806,
 201807,
 201808,
 201809,
 201810,
 201811,
 201812,
 201901,
 201902,
 201903]

In [5]:
uselog = pd.DataFrame()
for i in range(1,len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"] == year_months[i]]
    tmp.rename(columns = {"count" : "count_0"}, inplace = True)
    tmp_before = uselog_months.loc[uselog_months["年月"] == year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns = {"count" : "count_1"}, inplace = True)
    tmp2 = pd.merge(tmp, tmp_before, on  = "customer_id")
    uselog = pd.concat([uselog,tmp2], ignore_index = True) 
uselog

C:\Users\Takeshi\anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4
1,201805,AS009373,4,3
2,201805,AS015315,3,6
3,201805,AS015739,5,7
4,201805,AS015746,1,4
...,...,...,...,...
32645,201903,TS995853,8,11
32646,201903,TS998593,8,7
32647,201903,TS999079,3,2
32648,201903,TS999231,6,6


# ノック42(ここから自身の解答)

退会顧客が退会申請をした前月のデータを集計する。  
  
これは、退会申請をした1カ月後にend_dateが登録されるため。  
  
これで退会者の特徴を抽出する。  
Ex. end_dateが2018年8月の顧客は、2018年7月に退会申請をしている。そのため、その顧客の2018年6月のデータを見る。

In [6]:
# 退会した顧客データのみ集計

customer_enddate_only = customer.loc[~customer["end_date"].isnull()]
customer_enddate_only

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,ナイト,6000,通常,4.5,4.5,6,3,0,2019-02-28,1
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,通常,7.0,6.0,12,3,0,2019-03-31,2
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,デイタイム,7500,通常,7.0,7.0,11,3,0,2019-03-31,1
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,ナイト,6000,通常,7.0,7.0,10,4,0,2019-03-31,1


In [7]:
#end_dateの一ヶ月前の日付の列を作成。それを年月に変換して、退会申請をした月の列を作成。

customer_enddate_only["end_date"]  = pd.to_datetime(customer_enddate_only["end_date"])
customer_enddate_only["2nd_end_month"] = customer_enddate_only["end_date"] - datetime.timedelta(days = 31)
customer_enddate_only["2nd_end_month"] = customer_enddate_only["2nd_end_month"].dt.strftime("%Y%m")
customer_enddate_only

C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\Takeshi\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,2nd_end_month
708,TS511179,XXXXXX,C01,F,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-04-30,23,201803
729,TS443736,XXXX,C02,M,2016-05-01,2018-04-30,CA1,1,デイタイム,7500,通常,3.0,3.0,3,3,0,2018-04-30,23,201803
730,HD542886,XX,C01,M,2016-05-01,2018-04-30,CA1,1,オールタイム,10500,通常,1.0,1.0,1,1,0,2018-04-30,23,201803
770,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,通常,3.5,3.5,4,3,1,2018-05-31,23,201804
785,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,通常,3.0,3.0,3,3,0,2018-05-31,23,201804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4049,OA143589,XXXXX,C03,F,2019-01-15,2019-02-28,CA1,1,ナイト,6000,通常,4.5,4.5,6,3,0,2019-02-28,1,201901
4050,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,通常,7.0,6.0,12,3,0,2019-03-31,2,201902
4068,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,デイタイム,7500,通常,7.0,7.0,11,3,0,2019-03-31,1,201902
4086,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,ナイト,6000,通常,7.0,7.0,10,4,0,2019-03-31,1,201902


In [8]:
#uselogと結合

uselog_merge = pd.merge(customer_enddate_only, uselog, on = "customer_id")
uselog_merge

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,...,median,max,min,routine_flg,calc_date,membership_period,2nd_end_month,年月,count_0,count_1
0,HD597545,XXXXX,C03,F,2016-06-01,2018-05-31,CA1,1,ナイト,6000,...,3.5,4,3,1,2018-05-31,23,201804,201805,3,4
1,HI749296,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,...,3.0,3,3,0,2018-05-31,23,201804,201805,3,3
2,OA007936,XXXXX,C01,M,2016-06-01,2018-05-31,CA1,1,オールタイム,10500,...,3.0,4,2,0,2018-05-31,23,201804,201805,2,4
3,AS842977,XXXXX,C02,F,2016-06-01,2018-05-31,CA1,1,デイタイム,7500,...,4.0,5,3,1,2018-05-31,23,201804,201805,5,3
4,HD805772,XXXX,C02,F,2016-07-01,2018-05-31,CA2,1,デイタイム,7500,...,3.0,3,3,0,2018-05-31,22,201804,201805,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5911,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,...,6.0,12,3,0,2019-03-31,2,201902,201902,6,12
5912,GD367382,XXXX,C01,M,2019-01-15,2019-03-31,CA1,1,オールタイム,10500,...,6.0,12,3,0,2019-03-31,2,201902,201903,3,6
5913,OA034442,XXXXX,C02,M,2019-02-05,2019-03-31,CA1,1,デイタイム,7500,...,7.0,11,3,0,2019-03-31,1,201902,201903,3,11
5914,PL508278,XXXXX,C03,M,2019-02-09,2019-03-31,CA1,1,ナイト,6000,...,7.0,10,4,0,2019-03-31,1,201902,201903,4,10


In [9]:
#年月の列をobject型に変換。その後、2nd_end_monthと年月が合致する行だけを取り出す。

uselog_merge["年月"] = uselog_merge["年月"].astype(str)
uselog_merge_2ndendmonth =uselog_merge.loc[uselog_merge["2nd_end_month"] == uselog_merge["年月"]]
uselog_merge_2ndendmonth

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,...,median,max,min,routine_flg,calc_date,membership_period,2nd_end_month,年月,count_0,count_1
10,HD262908,XXXXX,C03,F,2016-08-01,2018-06-30,CA1,1,ナイト,6000,...,2.0,4,1,0,2018-06-30,22,201805,201805,4,2
16,HD944583,XXXX,C02,F,2016-09-01,2018-07-31,CA1,1,デイタイム,7500,...,4.0,6,3,1,2018-07-31,22,201806,201806,3,4
21,AS047907,XXX,C01,M,2016-10-01,2018-08-31,CA1,1,オールタイム,10500,...,4.0,5,3,1,2018-08-31,22,201807,201807,4,5
26,AS280744,XXX,C02,M,2016-10-01,2018-09-30,CA1,1,デイタイム,7500,...,4.0,4,1,1,2018-09-30,23,201808,201808,4,4
30,OA308827,XXX,C01,F,2016-10-01,2018-08-31,CA1,1,オールタイム,10500,...,2.0,3,1,0,2018-08-31,22,201807,201807,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5900,AS194818,XXXXX,C02,F,2019-01-05,2019-03-31,CA1,1,デイタイム,7500,...,4.0,8,1,1,2019-03-31,2,201902,201902,1,8
5902,PL739722,XXXXXX,C03,M,2019-01-07,2019-03-31,CA1,1,ナイト,6000,...,6.0,7,2,0,2019-03-31,2,201902,201902,6,7
5904,HI815924,XXXXX,C02,F,2019-01-09,2019-03-31,CA1,1,デイタイム,7500,...,5.0,9,2,0,2019-03-31,2,201902,201902,5,9
5907,AS484213,XXXXX,C02,F,2019-01-12,2019-03-31,CA1,1,デイタイム,7500,...,4.0,10,1,0,2019-03-31,2,201902,201902,4,10


In [10]:
#　顧客のユニーク数を調べる

len(uselog_merge_2ndendmonth["customer_id"].unique())

1052